In [2]:
import numpy as np
import pandas as pd
import string

In [133]:
pr_sigma_states = pd.read_csv("D:/Data Science/Coursera/Bioinformatics/Find mutation of DNA_RNA_Protein/Book8.csv")
pr_sigma_states.set_index('Unnamed: 0', inplace = True)
pr_sigma_states

,A,B
Unnamed: 0,,
A,0.377,0.623
B,0.260,0.740


In [128]:
Sigma = "ABABBBAAAA"

In [134]:
def get_transisions(Sigma):
    transitions = []
    for i in range(1, len(Sigma)):
        transitions.append(pr_sigma_states.loc[Sigma[i-1], Sigma[i]])
    pr_sigma = (1/2)*np.prod(transitions)
    return(pr_sigma)

In [135]:
get_transisions(Sigma)

0.0003849286917546758

In [136]:
pr_emis_states = pd.read_csv("D:/Data Science/Coursera/Bioinformatics/Find mutation of DNA_RNA_Protein/Book6.csv")
pr_emis_states

,Unnamed: 0,x,y,z
0,A,0.429,0.422,0.149
1,B,0.662,0.052,0.286


In [137]:
pr_emis_states.set_index('Unnamed: 0', inplace = True)

In [138]:
pr_emis_states

,x,y,z
Unnamed: 0,,,
A,0.429,0.422,0.149
B,0.662,0.052,0.286


In [139]:
Sigma = "BBAAAABBABBBBAAABBBBBBBBBAAAABBBAABBBABABBBAABAABB"
States = "yyzzyxzxyzxyxyzzyzyxyzyzxxxxzxxxzzzzxyxyxyxyxxzyzx"

In [140]:
def get_emission(Sigma, States):
    pr_emis = []
    for i in range(len(Sigma)):
        pr_emis.append(pr_emis_states.loc[Sigma[i], States[i]])
    pr_es = np.prod(pr_emis)
    return(pr_es)

In [141]:
emission(Sigma, States)

6.250728434673316e-29

# Profile HMMs for Sequence Alignment

In [87]:
alignment = []
#with open('D:/Data Science/Coursera/Bioinformatics/Find mutation of DNA_RNA_Protein/hmm_7.txt') as f:
with open('D:/Data Science/Coursera/Bioinformatics/Find mutation of DNA_RNA_Protein/hmm_8.txt') as f:
  while True:
    s = []
    c = f.readline().replace("\n", "")
    for x in c:
        s.append(x)
    if not c:
      break 
    alignment.append(s)
alignment = np.array(alignment)

In [38]:
alignment

array([['A', 'C', 'D', 'E', 'F', 'A', 'C', 'A', 'D', 'F'],
       ['A', 'F', 'D', 'A', '-', '-', '-', 'C', 'C', 'F'],
       ['A', '-', '-', 'E', 'F', 'D', '-', 'F', 'D', 'C'],
       ['A', 'C', 'A', 'E', 'F', '-', '-', 'A', '-', 'C'],
       ['A', 'D', 'D', 'E', 'F', 'A', 'A', 'A', 'D', 'F']], dtype='<U1')

In [88]:
N, D = alignment.shape
theta = 0.289

In [89]:
alphabet = list(string.ascii_uppercase)

In [41]:
def get_alignment_seed(alignment, theta):
    ignore = []
    index = [i for i in range(D)]
    for i in range(D):
        apear = alignment[:, i].tolist().count("-")
        prop = apear/N
        if prop > theta or prop == theta:
            ignore.append(i) 
    true = list(set(index).difference(set(ignore)))
    return(alignment[:, true], alignment[:, ignore], true, ignore)

In [90]:
N_seed = get_alignment_seed(alignment, theta)[0].shape[1]
N_seed

2

In [43]:
def get_states_alignment(alignment, theta):

    alignment_seed = get_alignment_seed(alignment, theta)[0]
    alignment_ignore = get_alignment_seed(alignment, theta)[1]
    true = get_alignment_seed(alignment, theta)[2]
    ignore = get_alignment_seed(alignment, theta)[3]
    st_al_df = pd.DataFrame({'S': np.zeros((N))})
    
    if ignore == []:
        st_al_df['I0'] = np.zeros((N))
    elif ignore != []:
        if ignore[0] == 0:
            st_al_df['I0'] = alignment_ignore[:, 0]
        elif ignore[0] != 0:
            st_al_df['I0'] = np.zeros((N))
        
    k = alignment_seed.shape[1]
    for i in range(k):
        st_al_df['M' + str(i+1)] = alignment_seed[:, i]
        st_al_df['D' + str(i+1)] = np.zeros(N) 
        
        ind = []
        for index in ignore:
            if i < k-1: 
                if index > true[i] and index < true[i+1]:
                    ind.append(index)
            else:
                if index > true[i]:
                    ind.append(index)
        
        if ind != []:
            col = []
            for j in range(N):
                x = alignment[j, ind]
                counting_alphabet = len(list(set(x).intersection(set(alphabet))))

                if counting_alphabet == len(x):
                    col.append("".join(x))
                elif counting_alphabet < len(x) and counting_alphabet > 0:
                    string = []
                    for y in x:
                        if y != "-":
                            string.append(y)
                    col.append("".join(string))
                elif counting_alphabet == 0:
                    col.append("-")

            st_al_df['I' + str(i+1)] = col
        else:
            st_al_df['I' + str(i+1)] = np.zeros((N))
         
    st_al_df["E"] = np.zeros((N))
    for i in range(1, alignment_seed.shape[1]+1):
        for j in range(N):
            if st_al_df['M' + str(i)][j] == "-":
                st_al_df['D' + str(i)][j] = "-"
    return(st_al_df)

In [91]:
states_alignment_df = get_states_alignment(alignment, theta)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [92]:
states_alignment_df

,S,I0,M1,D1,I1,M2,D2,I2,E
0,0.0,0.0,E,0.0,B,A,0,0.0,0.0
1,0.0,0.0,E,0.0,-,D,0,0.0,0.0
2,0.0,0.0,E,0.0,B,-,-,0.0,0.0
3,0.0,0.0,E,0.0,E,D,0,0.0,0.0
4,0.0,0.0,E,0.0,B,D,0,0.0,0.0
5,0.0,0.0,E,0.0,B,E,0,0.0,0.0
6,0.0,0.0,E,0.0,-,D,0,0.0,0.0
7,0.0,0.0,E,0.0,-,D,0,0.0,0.0


In [46]:
states_alignment_df.to_csv ("D:/Data Science/Coursera/Bioinformatics/Find mutation of DNA_RNA_Protein/states_alignment_df.csv", index = True, header=True)

In [93]:
sigma = ["A", "B", "C", "D", "E"]

In [94]:
def get_emission(alignment, theta, sigma):
    state_alignment_df = get_states_alignment(alignment, theta)
    columns = state_alignment_df.columns.values
    prop = []
    for x in columns:
        total = 0
        for t in state_alignment_df[x]:
            if t != "-" and t != 0:
                total += len(t)
        
        prop_sub = []
        for z in sigma:
            c = 0
            for t in state_alignment_df[x]:
                if t != 0:
                    c += t.count(z)
            if total != 0:
                prop_sub.append(c/total)  
            else:
                prop_sub.append(0)
        prop.append(prop_sub)  
    emisson = pd.DataFrame(np.array(prop), index = columns, columns = sigma)
    return(emisson)

In [95]:
def trans_prop(alignment, theta):
    
    alignment_seed = get_alignment_seed(alignment, theta)[0]
    state_alignment_df = get_states_alignment(alignment, theta)
    cols = state_alignment_df.columns.values
    m = len(cols)
    trans_prop = pd.DataFrame(np.zeros((m, m)), index = cols, columns = cols)
    end = alignment_seed.shape[1]
    
    prop_im = []
    prop_id = []
    prop_md = []
    prop_mi = []
    prop_mm = []
    prop_di = []
    prop_dm = []
    prop_dd = []
    prop_ii = []    
    prop_ie = []
    prop_de = []
    prop_me = []
    
    count_si = 0
    count_sd = 0
    count_sm = 0
    count_ie = 0
    count_de = 0
    count_me = 0
    count_mie = 0
    
    denom_iee = 0
    denom_dee = N
    denom_mee = N
    
    for x, y, z in zip(state_alignment_df['I0'], state_alignment_df['D1'], state_alignment_df['M1']):
        if x != 0 and x != '-':
            count_si += 1
        if y == "-":
            count_sd += 1
        if z != "-":
            count_sm += 1
            
        if x != "-" and x != 0:
            counting_alphabet = len(x)
            trans_prop.loc['I0', 'I0'] = (counting_alphabet-1)/N
        else:
            trans_prop.loc['I0', 'I0'] = 0
    prop_si = count_si/N
    prop_sd = count_sd/N
    prop_sm = count_sm/N
    
    trans_prop.loc['S', 'I0'] = prop_si
    trans_prop.loc['S', 'M1'] = prop_sm
    trans_prop.loc['S', 'D1'] = prop_sd
    
    for x, y, z in zip(state_alignment_df['I'+str(end)],
                       state_alignment_df['D'+str(end)],
                       state_alignment_df['M'+str(end)]):
        if x != 0 and x != '-':
            count_ie += 1
        if y == "-":
            count_de += 1
        if z != "-" and x == "-" or x == 0:
            count_me += 1
        elif z != "-" and x != "-" and x!= 0:
            count_mie += 1
        if x != "-" and x!= 0:
            denom_iee += len(x)
        if y != "-":
            denom_dee -= 1
        if z == "-":
            denom_mee -= 1
    if denom_iee != 0:        
        prop_ie = count_ie/denom_iee
    else:
        prop_ie = 0   
    if  denom_dee != 0:
        prop_de = count_de/denom_dee
    else:    
        prop_de = 0
    if denom_mee != 0:   
        prop_me = count_me/denom_mee
        prop_mie = count_mie/denom_mee
    else:
        prop_me = 0
    
    trans_prop.loc['I'+str(end), 'E'] = prop_ie
    trans_prop.loc['M'+str(end), 'E'] = prop_me
    trans_prop.loc['D'+str(end), 'E'] = prop_de 
    trans_prop.loc['M'+str(end), 'I'+str(end)] = prop_mie

    for i in range(1, end):
        count_ii = 0  
        count_im = 0
        count_id = 0
        count_md = 0
        count_mi = 0
        count_mm = 0
        count_di = 0
        count_dm = 0
        count_dd = 0
        denom_m = N
        denom_d = N
        denom_ii = 0
        denom_imd = 0

        for x, y, z, t in zip(state_alignment_df['M'+str(i)], 
                              state_alignment_df['D'+str(i+1)], 
                              state_alignment_df['I'+str(i)],
                              state_alignment_df['M'+str(i+1)]):
            
            if x != "-" and z !=0 and z!= "-":
                count_mi += 1    
            elif x != "-" and (z == 0 or z == "-"):     
                if t != "-":
                    count_mm += 1
                else:
                    count_md += 1
            if x == "-":
                denom_m -= 1
            
            if z != "-" and z != 0:
                count_ii += len(z) - 1
                denom_ii += len(z) 
        
        for x, y, z in zip(state_alignment_df['I'+str(i-1)], 
                           state_alignment_df['M'+str(i)], 
                           state_alignment_df['D'+str(i)]):
            if x != 0 and x != "-" and y!= "-":
                count_im += 1
            if x != 0 and x != "-" and z == "-":
                count_id += 1 
            if x != 0 and x != "-":
                denom_imd += len(x)
        if denom_imd != 0:        
            prop_im.append(count_im/denom_imd)
            prop_id.append(count_id/denom_imd)
        else: 
            prop_im.append(0)
            prop_id.append(0)
            
        if denom_m != 0:
            prop_md.append(count_md/denom_m)
            prop_mi.append(count_mi/denom_m)
            prop_mm.append(count_mm/denom_m)
        else:
            prop_md.append(0)
            prop_mi.append(0)
            prop_mm.append(0)
            
        if denom_ii != 0:
            prop_ii.append(count_ii/denom_ii)
        else:
            prop_ii.append(0)
            
        for x, y, z, t in zip(state_alignment_df['D'+str(i)], 
                              state_alignment_df['I'+str(i)], 
                              state_alignment_df['M'+str(i+1)],
                              state_alignment_df['D'+str(i+1)]):
            if x == "-" and y != 0 and y != "-":
                count_di += 1
            if x == "-" and z!= "-":
                count_dm += 1
            if x == "-" and t == "-":
                count_dd += 1
            if x != "-":
                denom_d -= 1

        if denom_d != 0:
            prop_di.append(count_di/denom_d) 
            prop_dm.append(count_dm/denom_d)
            prop_dd.append(count_dd/denom_d)
        else:
            prop_di.append(0)
            prop_dm.append(0)
            prop_dd.append(0)
        
        trans_prop.loc['I'+str(i), 'I'+str(i)] = prop_ii[i-1]
        trans_prop.loc['I'+str(i-1), 'D'+str(i)] = prop_id[i-1]
        trans_prop.loc['I'+str(i-1), 'M'+str(i)] = prop_im[i-1]
        trans_prop.loc['M'+str(i), 'D'+str(i+1)] = prop_md[i-1]
        trans_prop.loc['M'+str(i), 'I'+str(i)] = prop_mi[i-1]
        trans_prop.loc['M'+str(i), 'M'+str(i+1)] = prop_mm[i-1]
        trans_prop.loc['D'+str(i), 'I'+str(i)] = prop_di[i-1]
        trans_prop.loc['D'+str(i), 'D'+str(i+1)] = prop_dd[i-1]
        trans_prop.loc['D'+str(i), 'M'+str(i+1)] = prop_dm[i-1]
        
    count_ime = 0
    count_ide = 0
    count_mie = 0
    denom_ie = 0
    
    for x, y, z in zip( state_alignment_df['I'+str(end-1)], 
                        state_alignment_df['M'+str(end)], 
                        state_alignment_df['D'+str(end)]):
        if x != 0 and x != "-" and y!= "-":
            count_ime += 1
        if x != 0 and x != "-" and z == "-":
            count_ide += 1
        if x != "-" and x != 0:
            denom_ie += len(x)
            
    if denom_ie != 0:        
        prop_ime = count_ime/denom_ie
        prop_ide = count_ide/denom_ie
    else: 
        prop_ime = 0
        prop_ide = 0
        
    trans_prop.loc['I'+str(end-1), 'M'+str(end)] = prop_ime
    trans_prop.loc['I'+str(end-1), 'D'+str(end)] = prop_ide
    
    return(trans_prop)

In [96]:
emit_p= get_emission(alignment, theta, sigma)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [97]:
trans_p = trans_prop(alignment, theta)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [98]:
emit_p

,A,B,C,D,E
S,0.000000,0.0,0.0,0.000000,0.000000
I0,0.000000,0.0,0.0,0.000000,0.000000
M1,0.000000,0.0,0.0,0.000000,1.000000
D1,0.000000,0.0,0.0,0.000000,0.000000
I1,0.000000,0.8,0.0,0.000000,0.200000
M2,0.142857,0.0,0.0,0.714286,0.142857
D2,0.000000,0.0,0.0,0.000000,0.000000
I2,0.000000,0.0,0.0,0.000000,0.000000
E,0.000000,0.0,0.0,0.000000,0.000000


In [99]:
 trans_p                         

,S,I0,M1,D1,I1,M2,D2,I2,E
S,0.0,0.0,1.0,0.0,0.000,0.000,0.0,0.0,0.000000
I0,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.000000
M1,0.0,0.0,0.0,0.0,0.625,0.375,0.0,0.0,0.000000
D1,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.000000
I1,0.0,0.0,0.0,0.0,0.000,0.800,0.2,0.0,0.000000
M2,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,1.142857
D2,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,1.000000
I2,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.000000
E,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.000000


In [77]:
states = trans_p.columns.values
obs = ["A", "E", "F", "D", "F", "D", "C"]

In [78]:
states

array(['S', 'I0', 'M1', 'D1', 'I1', 'M2', 'D2', 'I2', 'M3', 'D3', 'I3',
       'M4', 'D4', 'I4', 'M5', 'D5', 'I5', 'M6', 'D6', 'I6', 'M7', 'D7',
       'I7', 'M8', 'D8', 'I8', 'E'], dtype=object)

In [79]:
obs

['A', 'E', 'F', 'D', 'F', 'D', 'C']

In [80]:
Sigma = ''.join(obs)

In [81]:
emis = emit_p
trans = trans_p

In [82]:
transition = {}
emission = {}

for x in trans.index:
    tr = {}
    for y in trans.columns:
        tr.update({y : trans.loc[x, y]})
    transition.update({x : tr})    
    
for x in emis.index:
    em = {}
    for y in emis.columns:
        em.update({y : emis.loc[x, y]})
    emission.update({x : em}) 

In [83]:
def get_viterbi_algorithm(Sigma, states, transition, emission):
    
    n =  len(Sigma)
    source_prop = 1/len(states)
    back_track = {}
    path = []
    s = {}
    
    # Finding s_{k, source}
    for k in states:
        s.update({k : {0 : source_prop*emission[k][Sigma[0]]}})
    # Finding s_{k, i}
    for i in range(1, n):
        sub_back_track = {}
        
        for k in states:    
            prop = []
            _emis = emission[k][Sigma[i]]
            
            for l in states:
                prop.append(s[l][i-1]*transition[l][k]*_emis)
               
            s[k].update({i : max(prop)})
            
            sub_back_track.update({k : states[np.argmax(prop)]})
        # Get back_tracking 
        back_track.update({i : sub_back_track})
    
    prop = []    
    
    # Get the last state.
    for l in states:
        prop.append(s[l][n-1])
    state = states[np.argmax(prop)]
    
    # Find the path of states.
    path.append(state)

    for i in reversed(range(1, n)):
        state = back_track[i][state] 
        path.append(state)
        
    return(''.join(reversed(path)))    

In [85]:
get_viterbi_algorithm(Sigma, states, transition, emission)

'M3M4M5I5M6M7M8'

In [86]:
states

array(['S', 'I0', 'M1', 'D1', 'I1', 'M2', 'D2', 'I2', 'M3', 'D3', 'I3',
       'M4', 'D4', 'I4', 'M5', 'D5', 'I5', 'M6', 'D6', 'I6', 'M7', 'D7',
       'I7', 'M8', 'D8', 'I8', 'E'], dtype=object)